<a href="https://colab.research.google.com/github/RoboticsLabURJC/2019-phd-sergio-paniego/blob/main/behavior_studio_networks/BS_recurrent_pilotnet_2_networks.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# When using Colab, check the GPU that is assigned and reload the runtime if its memory is low
!nvidia-smi

In [ ]:
# Mount Google Drive to access images dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive"
!ls "/content/drive/My Drive/complete_dataset.zip"
!unzip "/content/drive/My Drive/curves_only.zip"
!unzip "/content/drive/My Drive/complete_dataset.zip"

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Dropout, ConvLSTM2D, Reshape, Activation, MaxPooling2D
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam


def pilotnet_model(img_shape):
    '''
    Model of End to End Learning for Self-Driving Cars (NVIDIA)
    '''
    model = Sequential()
    model.add(BatchNormalization(epsilon=0.001, axis=-1, input_shape=img_shape))
    model.add(Conv2D(24, (5, 5), strides=(2, 2), activation="relu",padding='same'))
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation="relu",padding='same'))
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation="relu",padding='same'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), activation="relu",padding='same'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), activation="relu",padding='same'))
    model.add(Flatten())
    model.add(Dense(1164, activation="relu"))
    model.add(Dense(100, activation="relu"))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(1))
    adam = Adam(lr=0.0001)
    model.compile(optimizer=adam, loss="mse", metrics=['accuracy', 'mse', 'mae'])
    return model


def tinypilotnet_model(img_shape):
    model = Sequential()
    model.add(BatchNormalization(epsilon=0.001, axis=-1, input_shape=img_shape))
    model.add(Conv2D(8, (3, 3), strides=(2, 2), activation="relu"))
    model.add(Conv2D(8, (3, 3), strides=(2, 2), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(50, activation="relu"))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(1))
    adam = Adam(lr=0.0001)
    model.compile(optimizer=adam, loss="mse", metrics=['accuracy', 'mse', 'mae'])
    return model


In [ ]:
import glob
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

def load_data(folder):
    name_folder = '/content/' + folder + '/Images/'
    list_images = glob.glob(name_folder + '*')
    print(list_images)
    images = sorted(list_images, key=lambda x: int(x.split('/')[4].split('.png')[0]))
    name_file = '/content/' + folder + '/data.json'
    file = open(name_file, 'r')
    data = file.read()
    file.close()
    return images, data

def get_images(list_images, type_image, array_imgs):
    # Read the images
    for name in list_images:
        img = cv2.imread(name)
        if type_image == 'cropped':
            img = img[240:480, 0:640]
        img = cv2.resize(img, (int(img.shape[1] / 4), int(img.shape[0] / 4)))
        array_imgs.append(img)

    return array_imgs

def parse_json(data, array_v, array_w):
    # Process json
    data_parse = data.split('}')[:-1]
    for d in data_parse:
        v = d.split('"v": ')[1]
        d_parse = d.split(', "v":')[0]
        w = d_parse.split(('"w": '))[1]
        array_v.append(float(v))
        array_w.append(float(w))

    return array_v, array_w

def preprocess_data(array_w, array_v, imgs):
    # Take the image and just flip it and negate the measurement
    flip_imgs = []
    array_flip_w = []
    for i in range(len(array_w)):
        flip_imgs.append(cv2.flip(imgs[i], 1))
        array_flip_w.append(-array_w[i])
    new_array_w = array_w + array_flip_w
    new_array_v = array_v + array_v
    new_array_imgs = imgs + flip_imgs
    return new_array_w, new_array_v, new_array_imgs

def add_extreme_data(array_w, imgs_w, array_v, imgs_v):
    for i in range(0, len(array_w)):
        if abs(array_w[i]) >= 1:
            if abs(array_w[i]) >= 2:
                num_iter = 10
            else:
                num_iter = 5
            for j in range(0, num_iter):
                array_w.append(array_w[i])
                imgs_w.append(imgs_w[i])
        if float(array_v[i]) <= 2:
            for j in range(0, 1):
                array_v.append(array_v[i])
                imgs_v.append(imgs_v[i])
    return array_w, imgs_w, array_v, imgs_v


# Load data
images, data = load_data('complete_dataset')
images_curve, data_curve = load_data('curves_only')

# CHANGE type_image
type_image = 'cropped'
#type_image='normal'

# Preprocess images
array_imgs = []
x = get_images(images, type_image, array_imgs)
x = get_images(images_curve, type_image, x)
# Preprocess json
array_v = []
array_w = []
y_v, y_w = parse_json(data, array_v, array_w)
y_v, y_w = parse_json(data_curve, y_v, y_w)



if type_image == 'cropped':
    img_shape = (65, 160, 3)
else:
    img_shape = (120, 160, 3)

# Adapt the data
y_w, y_v, x = preprocess_data(y_w, y_v, x)
x_w = x[:]
x_v = x[:]
y_w, x_w, y_v, x_v = add_extreme_data(y_w, x_w, y_v, x_v)
X_train_v, X_validation_v, y_train_v, y_validation_v = train_test_split(x_v, y_v, test_size=0.20, random_state=42)
X_train_w, X_validation_w, y_train_w, y_validation_w = train_test_split(x_w, y_w, test_size=0.20, random_state=42)


# Adapt the data
X_train_v = np.stack(X_train_v, axis=0)
y_train_v = np.stack(y_train_v, axis=0)
X_validation_v = np.stack(X_validation_v, axis=0)
y_validation_v = np.stack(y_validation_v, axis=0)

X_train_w = np.stack(X_train_w, axis=0)
y_train_w = np.stack(y_train_w, axis=0)
X_validation_w = np.stack(X_validation_w, axis=0)
y_validation_w = np.stack(y_validation_w, axis=0)



In [ ]:
print(X_train_v[1].shape)
print(img_shape)
img_shape = (60, 160, 3)
print(img_shape)

model_v = pilotnet_model(img_shape)
model_w = pilotnet_model(img_shape)
# model_v = tinypilotnet_model(img_shape)
# model_w = tinypilotnet_model(img_shape)
batch_size_v = 64  # 16
batch_size_w = 64
nb_epoch_v = 300  # 223
nb_epoch_w = 300  # 212

if type_image == 'cropped':
    model_file_v = '/content/drive/My Drive/model_pilotnet_cropped_300_v.h5'
    model_file_w = '/content/drive/My Drive/model_pilotnet_cropped_300_w.h5'
else:
    model_file_v = '/content/drive/My Drive/model_pilotnet_v.h5'
    model_file_w = '/content/drive/My Drive/model_pilotnet_w.h5'

# Print layers
print(model_v.summary())

#  Train
model_history_v = model_v.fit(X_train_v, y_train_v, epochs=nb_epoch_v, batch_size=batch_size_v, verbose=2, validation_data=(X_validation_v, y_validation_v), callbacks=[])
# Save the model V
model_v.save(model_file_v)

#  Train
model_history_w = model_w.fit(X_train_w, y_train_w, epochs=nb_epoch_w, batch_size=batch_size_w, verbose=2, validation_data=(X_validation_w, y_validation_w), callbacks=[])
# Save the model W
model_w.save(model_file_w)


# We evaluate the model
score = model_v.evaluate(X_validation_v, y_validation_v, verbose=0)
print('Evaluating v')
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])
print('Test mean squared error: ', score[2])
print('Test mean absolute error: ', score[3])

score = model_w.evaluate(X_validation_w, y_validation_w, verbose=0)
print('Evaluating w')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test mean squared error: ', score[2])
print('Test mean absolute error: ', score[3])

